<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_Download_Archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
import requests
import re
from unicodedata import normalize
import os
from zipfile import ZipFile
from google.colab import files
from IPython.display import HTML, display, clear_output

In [ ]:
#@title Enter GetAccept login information
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
BASEURL = "https://api.getaccept.com/v1" #@param {type:"string"}
SOURCE_ENTITY = "" #@param {type: "string"}
DOCUMENT_COUNT = 100 #@param {type: "number"}
DOCUMENT_OFFSET = 0 #@param {type: "number"}

In [ ]:
# Make auth request and store auth header
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": SOURCE_ENTITY }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))
else:
  raise Exception("Please provide email and password to the GetAccept entity you want to export documents from")

In [ ]:
# Get data for signed documents
url = BASEURL + "/documents"
params = { "filter": "signed", "showall": "true", "limit": DOCUMENT_COUNT, "offset": DOCUMENT_OFFSET }
response = requests.get(url, headers=source_auth_headers, params=params)
documents = response.json()

In [ ]:
# Define generic functions
def download_document(document_id, filename):
  url = "https://api.getaccept.com/v1/documents/" + document_id + "/download"
  headers = { "Authorization": "bearer "+access_token }
  params = { "direct": "true" }
  response = requests.get(url, headers=headers, params=params)
  directory = "downloads/"
  if response.content:
    if not os.path.exists(directory):
      os.makedirs(directory)
    with open(directory + filename, 'wb') as f:
      f.write(response.content)
      f.close()
    return directory + filename
  else:
    return None
 
def slugify(text, delim='-'):
    """Generates an slightly worse ASCII-only slug."""
    result = []
    punct_re = re.compile(r'[\t !"#$%&\'()*\-/<=>?@\[\\\]^_`{|},.:]+')
    for word in punct_re.split(text.lower()):
        word = normalize('NFKD', word).encode('ascii', 'ignore')
        word = word.decode('utf-8')
        if word:
            result.append(word)
    return delim.join(result)

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
# Export and download ZIP file with signed documents
zipfilename = "download-" + str(DOCUMENT_OFFSET) + "-" + str(DOCUMENT_OFFSET + DOCUMENT_COUNT) + ".zip"
count = 0
prog = display(progress(count, DOCUMENT_COUNT), display_id=True)
with ZipFile(zipfilename, "w") as zf:
  for document in documents:
    filename = slugify(document["name"]) + "_" + document["id"] + ".pdf"
    dlfile = download_document(document["id"],filename)
    zf.write(dlfile)
    os.remove(dlfile)
    count = count + 1
    prog.update(progress(count, DOCUMENT_COUNT))
zf.close()
files.download(zipfilename)